In [52]:
# Source: https://github.com/prestinomills/aqueduct/blob/Know_Your_Community_Pipelines/civis/geohub/ActiveBusinessBlockgroupAggregation/Active_Business_Finalized_Script.py
# %load active_business_script.py
"""
Created on Wed May  1 08:51:03 2019
@author: myrfid041
"""
import geopandas as gpd
import os
import pandas as pd

import esri_credentials

from sodapy import Socrata
from arcgis.gis import GIS
from arcgis.features.summarize_data import join_features
from IPython.display import display
from arcgis.features import FeatureLayer, FeatureLayerCollection, FeatureCollection, SpatialDataFrame


#---Setting the Outputs
OUTPUT_FILE = "./Listing_of_Active_Businesses.csv"
output_layer_name = "a7236cc62ded454c94a64e9d80d6304a"
max_record_count = 250_000

df = pd.read_pickle("./cleaned_df.p")

lahub_user = esri_credentials.preston_user
lahub_pass = esri_credentials.preston_pw

In [54]:
gis = GIS('https://lahub.maps.arcgis.com',  username=lahub_user, password=lahub_pass)

flayer = gis.content.get(output_layer_name)

ActiveBusinesses_flayer = flayer.layers[0].query()
existing_table = ActiveBusinesses_flayer.sdf

In [11]:
updated_table = pd.merge(
    existing_table[["OBJECTID", "GEOID10", "SHAPE"]],
    # Rename columns to match, drop columns not in layer
    # ESRI renamed the geometry to be SHAPE...and it's stored slightly different than in geodataframe
    (df.drop(columns = ["geometry"])
    ), 
    on = "GEOID10", 
    how = "inner")

In [93]:
new_updated_table = df[["GEOID10"] + list(layer_rename_columns_dict.values())]

,GEOID10,Accommodation_and_Food_Services,Administrative_and_Support_and_,Agriculture__Forestry__Fishing_,Arts__Entertainment__and_Recrea,Construction,Educational_Services,Finance_and_Insurance,Health_Care_and_Social_Assistan,Information,...,Medical_Marijuana_Collective,Mining,Not_Classified,Other_Services__except_Public_A,Professional__Scientific__and_T,Real_Estate_Rental_and_Leasing,Retail_Trade,Transportation_and_Warehousing,Utilities,Wholesale_Trade
0,060372932023,19,17,1,1,17,1,1,4,2,...,5,0,1,27,8,58,33,6,0,34
1,060372941201,0,0,0,0,0,0,0,0,0,...,0,0,0,3,2,2,3,5,0,4
2,060372932022,1,12,1,1,2,0,0,1,1,...,0,0,1,8,2,29,4,2,0,2
3,060372932021,0,2,0,1,5,0,0,1,1,...,0,0,1,1,6,7,4,0,0,2
4,060372941101,0,8,1,1,3,0,0,1,0,...,0,0,0,8,2,9,2,1,0,0


In [20]:
feature_list = ActiveBusinesses_flayer.features

In [31]:
#ActiveBusinesses_flayer.fields

In [30]:
table_as_dict = ActiveBusinesses_flayer.to_dict()

In [40]:
for key, value in table_as_dict.items():
    print(key)
    #print(table_as_dict[key][0])

features
objectIdFieldName
globalIdFieldName
spatialReference
geometryType
fields


In [94]:
test_table = new_updated_table.head(3)
test_table

,GEOID10,Accommodation_and_Food_Services,Administrative_and_Support_and_,Agriculture__Forestry__Fishing_,Arts__Entertainment__and_Recrea,Construction,Educational_Services,Finance_and_Insurance,Health_Care_and_Social_Assistan,Information,...,Medical_Marijuana_Collective,Mining,Not_Classified,Other_Services__except_Public_A,Professional__Scientific__and_T,Real_Estate_Rental_and_Leasing,Retail_Trade,Transportation_and_Warehousing,Utilities,Wholesale_Trade
0,060372932023,19,17,1,1,17,1,1,4,2,...,5,0,1,27,8,58,33,6,0,34
1,060372941201,0,0,0,0,0,0,0,0,0,...,0,0,0,3,2,2,3,5,0,4
2,060372932022,1,12,1,1,2,0,0,1,1,...,0,0,1,8,2,29,4,2,0,2


In [95]:
test_dict = test_table.set_index("GEOID10").to_dict(orient="index")
test_dict.keys()

dict_keys(['060372932023', '060372941201', '060372932022'])

In [102]:
for k, v in test_dict['060372932023'].items():
    print(k)

Accommodation_and_Food_Services
Administrative_and_Support_and_
Agriculture__Forestry__Fishing_
Arts__Entertainment__and_Recrea
Construction
Educational_Services
Finance_and_Insurance
Health_Care_and_Social_Assistan
Information
Manufacturing
Medical_Marijuana_Collective
Mining
Not_Classified
Other_Services__except_Public_A
Professional__Scientific__and_T
Real_Estate_Rental_and_Leasing
Retail_Trade
Transportation_and_Warehousing
Utilities
Wholesale_Trade


In [101]:
test_dict['060372932023']["Accommodation_and_Food_Services"]

19

In [96]:
for k, v in test_dict["060372932023"].items():
    print(v)

19
17
1
1
17
1
1
4
2
10
5
0
1
27
8
58
33
6
0
34


In [60]:
for i in range(0, 2):
    print(f"*****Obs: {i}*****")
    obs = table_as_dict["features"][i]
    print("Keys")
    print(obs["attributes"].keys())
    print("Values")
    print(obs["attributes"].values())

*****Obs: 0*****
Keys
dict_keys(['OBJECTID', 'GEOID10', 'CTBG10', 'CT10', 'AreaSqMil', 'LABEL', 'FIP10', 'FIP10RV', 'CDP_NAME', 'CITYNAME', 'COMMNAME', 'Accommodation_and_Food_Services', 'Administrative_and_Support_and_', 'Agriculture__Forestry__Fishing_', 'Arts__Entertainment__and_Recrea', 'Construction', 'Educational_Services', 'Finance_and_Insurance', 'Health_Care_and_Social_Assistan', 'Information', 'Manufacturing', 'Medical_Marijuana_Collective', 'Mining', 'Not_Classified', 'Other_Services__except_Public_A', 'Professional__Scientific__and_T', 'Real_Estate_Rental_and_Leasing', 'Retail_Trade', 'Transportation_and_Warehousing', 'Utilities', 'Wholesale_Trade', 'SHAPE'])
Values
dict_values([1, '060372932023', '2932023', '293202', 0.2272, 'BG 3', '44000', '44000', 'Los Angeles', 'Los Angeles', 'LA City - Harbor City', 9, 20, 0, 22, 19, 2, 3, 19, 7, 6, 0, 0, 0, 21, 21, 21, 26, 6, 0, 25, {'rings': [[[6470819.15699448, 1756917.22135057], [6470821.72883973, 1756760.7246163], [6470823.804951

In [51]:
table_as_dict["features"][10]["attributes"]["GEOID10"]

'060371033001'

In [46]:
table_as_dict["fields"]

[{'alias': 'OBJECTID',
  'defaultValue': None,
  'domain': None,
  'name': 'OBJECTID',
  'sqlType': 'sqlTypeInteger',
  'type': 'esriFieldTypeOID'},
 {'alias': 'GEOID10',
  'defaultValue': None,
  'domain': None,
  'length': 12,
  'name': 'GEOID10',
  'sqlType': 'sqlTypeNVarchar',
  'type': 'esriFieldTypeString'},
 {'alias': 'CTBG10',
  'defaultValue': None,
  'domain': None,
  'length': 7,
  'name': 'CTBG10',
  'sqlType': 'sqlTypeNVarchar',
  'type': 'esriFieldTypeString'},
 {'alias': 'CT10',
  'defaultValue': None,
  'domain': None,
  'length': 6,
  'name': 'CT10',
  'sqlType': 'sqlTypeNVarchar',
  'type': 'esriFieldTypeString'},
 {'alias': 'AreaSqMil',
  'defaultValue': None,
  'domain': None,
  'name': 'AreaSqMil',
  'sqlType': 'sqlTypeDecimal',
  'type': 'esriFieldTypeDouble'},
 {'alias': 'LABEL',
  'defaultValue': None,
  'domain': None,
  'length': 4,
  'name': 'LABEL',
  'sqlType': 'sqlTypeNVarchar',
  'type': 'esriFieldTypeString'},
 {'alias': 'FIP10',
  'defaultValue': None,


In [ ]:
activebiz_flayer = flayer.layers[0]
layer_fset = activebiz_flayer.query()
features_for_update = [] #list containing corrected features
all_features = layer_fset.features
original_feature = [f for f in all_features if f.attributes['GEOID10'] == GEOID10][0]


In [ ]:
item_props = {'title' : 'Active Businesses TEST', 'description': 'here is description'}
flayer.update(item_properties = item_props, data = "./updated_data.csv")

In [ ]:
# Upload new item to begin with
#item_prop = {"title": "ActiveBizTest"}
#gis.content.add(item_properties=item_prop, data="./upload_me.csv")

In [ ]:
activebiz_flayer = flayer.layers[0]
layer_fset = activebiz_flayer.query()
features_for_update = [] #list containing corrected features
all_features = layer_fset.features

In [87]:
layer_rename_columns_dict = {
    'Accommodation and Food Services': 'Accommodation_and_Food_Services',
    'Administrative and Support and Waste Management and Remediation Services': 'Administrative_and_Support_and_',
    'Agriculture, Forestry, Fishing and Hunting': 'Agriculture__Forestry__Fishing_',
    'Arts, Entertainment, and Recreation': 'Arts__Entertainment__and_Recrea',
    'Construction': 'Construction',
    'Educational Services': 'Educational_Services',
    'Finance and Insurance': 'Finance_and_Insurance',
    'Health Care and Social Assistance': 'Health_Care_and_Social_Assistan',
    'Information': 'Information',
    'Manufacturing': 'Manufacturing',
    'Medical Marijuana Collective': 'Medical_Marijuana_Collective',
    'Mining': 'Mining',
    'Not Classified': 'Not_Classified',
    'Other Services (except Public Administration)': 'Other_Services__except_Public_A',
    'Professional, Scientific, and Technical Services': 'Professional__Scientific__and_T',
    'Real Estate Rental and Leasing': 'Real_Estate_Rental_and_Leasing',
    'Retail Trade': 'Retail_Trade',
    'Transportation and Warehousing': 'Transportation_and_Warehousing',
    'Utilities': 'Utilities',
    'Wholesale Trade': 'Wholesale_Trade'                                               
}

In [ ]:
wrong_geoid = ["060375722021"]
overlap_rows[overlap_rows.GEOID10.isin(wrong_geoid)]

In [ ]:
df[df.GEOID10.isin(wrong_geoid)]

In [ ]:
#https://developers.arcgis.com/python/sample-notebooks/updating-features-in-a-feature-layer/
from copy import deepcopy

overlap_rows = df[df.GEOID10.isin(layer_fset.sdf.GEOID10)]
overlap_rows.to_csv("./upload_csv", index=False)

csv_table = pd.read_csv("./upload_me.csv", dtype = {"GEOID10": "str"})

for GEOID10 in overlap_rows['GEOID10']:
    print(GEOID10)
    # get the feature to be updated
    original_feature = [f for f in all_features if f.attributes['GEOID10'] == GEOID10][0]
    feature_to_be_updated = deepcopy(original_feature)

    # get the matching row from csv
    matching_row = csv_table.where(overlap_rows.GEOID10 == GEOID10).dropna()
    print("Matching Row")
    display(matching_row)
    # assign the updated values
    
    for col in layer_rename_columns_dict.values():
        try:
            feature_to_be_updated.attributes[col] = matching_row[col].values[0]
        except:
            print(GEOID10)
            print(col)
            display(feature_to_be_updated.attributes[col])
            display(matching_row[col])

In [ ]:
flayer_collection.manager.update_definition({"maxRecordCount": max_record_count})